[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_tensor_rt.ipynb)

In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!sudo apt-get update
!sudo apt-get install tensorrt
!sudo apt-get install python3-libnvinfer-dev
!sudo apt-get install uff-converter-tf graphsurgeon-tf onnx-graphsurgeon

!pip install pycuda nvidia-tensorrt==8.4.1.5

In [ ]:
!nvcc --version

In [ ]:
!cat /etc/issue
import tensorflow as tf
print(tf.__version__)

In [ ]:
!dpkg -l | grep libcudnn

In [ ]:
!dpkg -l | grep TensorRT

In [ ]:
!pip show pycuda

In [ ]:
!pip show nvidia-tensorrt

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

dest = numpy.zeros_like(a)
multiply_them(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1), grid=(1,1))

print(dest-a*b)

In [ ]:
!git clone https://github.com/stochasticai/x-stable-diffusion

In [ ]:
!pip install transformers diffusers scipy uvicorn pydantic fastapi pycuda huggingface_hub

In [ ]:
!git lfs install 

In [ ]:
!git clone -b onnx https://huggingface.co/runwayml/stable-diffusion-v1-5

In [ ]:
!python /content/x-stable-diffusion/TensorRT/convert_unet_to_tensorrt.py --onnx_unet_path /content/stable-diffusion-v1-4/unet/model.onnx --save_path /content/unet.engine

In [ ]:
!wget https://huggingface.co/kamalkraj/stable-diffusion-v1-4-onnx/resolve/main/models.tar.gz

In [ ]:
!tar -xf models.tar.gz

In [ ]:
!python /content/x-stable-diffusion/TensorRT/convert_unet_to_tensorrt.py

In [ ]:
!python /content/x-stable-diffusion/TensorRT/demo.py

In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
!wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.ckpt

In [ ]:
!python /content/diffusers/scripts/convert_stable_diffusion_checkpoint_to_onnx.py --model_path Anything-V3.0-pruned-fp16.ckpt --output_path /content/onnx --fp16

In [ ]:
!pip install onnx

In [ ]:
!git clone -b onnx https://huggingface.co/CompVis/stable-diffusion-v1-4

In [ ]:
!pip install huggingface-hub

In [29]:
from huggingface_hub import create_repo, create_branch, upload_folder, upload_file

In [ ]:
create_repo("embed/sd14e", token="token") 
upload_file(path_or_fileobj="/content/unet.engine", path_in_repo="unet.engine", repo_id="embed/sd14e", commit_message=f"unet.engine", token="token")

In [34]:
!mv /content/stable-diffusion-v1-4 /content/stable-diffusion-v1-4-old